In [0]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Imports

In [0]:
import numpy as np
import pandas as pd
import requests

In [0]:
import os
import json
from copy import deepcopy

In [0]:
logging.basicConfig?

In [0]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.getLogger("requests").setLevel(logging.ERROR) # silencing requests logging 

# Logging for this notebook
logger = logging.getLogger()
logger.setLevel(logging.DEBUG) # set this to whatever you'd like

# Introduction to the new FEC API

Making data accessible through the a modern API is a huge move forward for the Federal Elections Commission. With the help of a team of intrepid 18f developers, FEC is re-thinking both its website and its data offerings to better serve its mission of educating the public with real-time disclosure of campaign finance information. It's part of the larger OpenFEC project, and we think it's a very encouraging sign that this collaboration is going to improve access to a crucial information resource.

What follows is meant as both an introduction to what's available through this new resource and a critique of what's working well, and the changes Sunlight would like to see in future releases of the API.

## Doesn't the FEC already release data?

FEC is a model disclosure authority: it has made federal campaign finance data available through a searchable web portal, in bulk csv files, and, most impressively, a live feed of submitted disclosures. On Influence Explorer, we've made use of each of these sources in different ways - most recently turning that live feed into a searchable data resource at http://realtime.influenceexplorer.com.

Sunlight has consistently called on government sources to make all data available in bulk. It's difficult to know how a dataset might be used by a researcher, reporter, citizen or advocacy group, so it's important that government bodies release ALL of it in machine-readable bulk files to allow the fullest exploration of what's available and to give context to any given data point. The FEC has historically set an excellent example in making bulk data available. 

## Additional benefits from an API

We think pretty highly of what the FEC already offers, and encourage them to continue to make both bulk and streamed data available. Here at Sunlight, though, we tend to make the data we release available both in bulk and through APIs, because we think that APIs are the right kind of access for particular users and usecases. So what additional advantages are offered by an API?

### Selective data views

Not every user or developer can effectively make use of bulk data. It typically doesn't fit in an spreadsheet, so the point-and-click crowd can be at a loss right away. Even if you're technically skilled enough to load it into R or pandas, though, you may hit a barrier if the operations you want to carry out require that the data be loaded into memory. 

Furthermore, a bulk release may contain a lot of data that isn't relevant to a particular usecase or investigation. Let's say I want to look at contributions to House candidates who are Democrats in 2012. If I use the bulk release, I'm going to get a lot of data that's not interesting to me, including all of the contributions to non-candidate PACs, members of other parties, and contributions to presidential or senatorial candidates.

Pulling out exactly what you need usually requires loading everything into a database, setting up some indices, and running queries. True, there are some tools for working directly with CSV files, like the excellent [csvkit](https://csvkit.readthedocs.org/en/0.9.1/), but depending on your query, you again might run into memory issues. Good old \*nix standbys `grep`, `cut`, `sed` and `awk` can also get you pretty far, if you're willing to hone your shell scripts.

With an API, though, you can hand off this selection business to the data's source (in this case, FEC's data warehouse). As long as the API supports it, you can formulate your query and retrieve it with confidence. That's an important qualifier, though: The onus is on the API's design team to make sure that the views which are offered meet the needs of its likely users.

### Aggregate views

Another advantage to having an API is the ability to show up-to-date aggregations of the records in your database. This includes totals, rankings, and metadata that might change over time. 

Again, aggregation is something that anyone can perform on bulk data. That is, anyone with the technical know-how discussed in the last section, and the domain knowledge needed to properly compute aggregates. 

In the case of FEC data in particular, summing the dollar amounts of individual transactions can be deceptively difficult. Whether or not two amounts can be added together depends on the type of committee, the type of transaction, and sometimes also the type of contributor.

A certain level of legal and regulatory expertise is needed when calculating these sums, but might be out of scope for a developer that wants to add or explore some summary statistics from campaign finance, but for whom campaign finance is not the main focus of investigation. Maybe the focus is on projected vote share, and the campaign finance information is going to be added for context. In cases like that, it's useful to source the aggregate totals published by the FEC itself, and an API is a great way to deliver that data.

### Live data

In addition to being more focused and infused with expertise, data views made available through an API can be tied to live data. In the case of the FEC's new API, the data is updated daily. This partially avoids the need for a developer to repeatedly update their database with successive bulk data releases.

In fact, for some usecases, an API might obviate the need for a database altogether.  Imagine again the case of a website that shows some other, non-campaign-finance data, such as legislative activity or election results. If campaign finance data could be a helpful addition to that kind of app, the developer can avoid having to build a big addition onto their database by making client-side calls with javascript. The site, which may be backed by a large database, can deliver data in a web app, and then obtain FEC's aggregate totals or summary facts on-the-fly, allowing them to show up if and when the site's designers choose.

## Exploring the API

The base URL for the API is

In [0]:
BASE_URL = 'http://api.open.fec.gov/v1'

You'll also need a data.gov API key, which you can obtain [here](https://api.data.gov/signup/). I save my API keys in a plain text file in my home directory, so that they're always handy and so that I can use them without revealing them in notebooks like this one.

In [0]:
API_KEY = open(os.path.expanduser('~/.api-keys/data.gov'),'r').read().strip()

Conceptually, there are 2 main areas of focus for the API: Candidates and Committees. When looking at contributions, however, remember that recipients are always committees. Candidates do not recieve contributions directly: their committees do. Here are the relevant branches:

  - `/candidate`: individual candidate information
  - `/committee`: individual committee information

### Helpful Utils

Some methods and global vars to help us stay succinct below.

In [0]:
def all_results(endpoint, params):
    _params = deepcopy(params)
    _params.update({'api_key': API_KEY})
    _url = BASE_URL+endpoint
    logging.info('querying endpoint: {}'.format(_url))
    
    initial_resp = requests.get(_url, params=_params)
    
    logging.debug('full url eg: {}'.format(initial_resp.url))
            
    initial_data = initial_resp.json()
    
    num_pages = initial_data['pagination']['pages']
    num_records = initial_data['pagination']['count']
    logging.info('{p} pages to be retrieved, with {n} records'.format(
            p=num_pages, n=num_records))
    
    current_page = initial_data['pagination']['page']
    logging.debug('page {} retrieved'.format(current_page))
    
    for record in initial_data['results']:
        yield record
    
    while current_page < num_pages:
        current_page += 1
        _params.update({'page': current_page})
        _data = requests.get(_url, params=_params).json()
        logging.debug('page {} retrieved'.format(current_page))
        for record in _data['results']:
            yield record
            
    logging.info('all pages retrieved')
    
def count_results(endpoint, params):
    _params = deepcopy(params)
    _params.update({'api_key': API_KEY})
    _url = BASE_URL+endpoint
    
    _data = requests.get(_url, params=_params).json()
    
    return _data['pagination']['count']

### FEC Identifiers, via search or list

To get data associated with a candidate or a committee, you need to know the identifier that FEC has assigned to that entity. In case you don't have those memorized, though, there are two ways to obtain the IDs that you need.

The first is to simply list all of the candidates. This can be done at the `/candidates` endpoint. The metadata in the records returned can help when building a local reference resource or lookup table.

In [0]:
q_all_2012_candidates = {
    "cycle": 2012,
}

This query is going to return a LOT of candidates:

In [0]:
count_results('/candidates', q_all_2012_candidates)

3022

You can limit the list by specifying the `candidate_status`. Most of the time, what we care about are candidates with `candidate_status=C`, which means they are a declared candidate who has raised at least $5,000 in that cycle. 

In [0]:
q_all_2012_present_candidates = {
    "cycle": 2012,
    "candidate_status": "C"
}

In [0]:
count_results('/candidates', q_all_2012_present_candidates)

728

That's a little more like it. Now we can pull that data down and see how it looks.

In [0]:
candidates_2012 = [c for c in all_results('/candidates', q_all_2012_candidates)]

INFO:root:querying endpoint: http://api.open.fec.gov/v1/candidates
DEBUG:root:full url eg: http://api.open.fec.gov/v1/candidates?api_key=GLIbeNxQRAO4Z5S7buBeMpxiX7AopoSbi5hqSZBI&cycle=2012
INFO:root:152 pages to be retrieved, with 3022 records
DEBUG:root:page 1 retrieved
DEBUG:root:page 2 retrieved
DEBUG:root:page 3 retrieved
DEBUG:root:page 4 retrieved
DEBUG:root:page 5 retrieved
DEBUG:root:page 6 retrieved
DEBUG:root:page 7 retrieved
DEBUG:root:page 8 retrieved
DEBUG:root:page 9 retrieved
DEBUG:root:page 10 retrieved
DEBUG:root:page 11 retrieved
DEBUG:root:page 12 retrieved
DEBUG:root:page 13 retrieved
DEBUG:root:page 14 retrieved
DEBUG:root:page 15 retrieved
DEBUG:root:page 16 retrieved
DEBUG:root:page 17 retrieved
DEBUG:root:page 18 retrieved
DEBUG:root:page 19 retrieved
DEBUG:root:page 20 retrieved
DEBUG:root:page 21 retrieved
DEBUG:root:page 22 retrieved
DEBUG:root:page 23 retrieved
DEBUG:root:page 24 retrieved
DEBUG:root:page 25 retrieved
DEBUG:root:page 26 retrieved
DEBUG:root:

In [0]:
[c for c in candidates_2012 if 'OBAMA' in c['name']]

[{u'active_through': 2012,
  u'candidate_id': u'P80003338',
  u'candidate_status': u'N',
  u'candidate_status_full': u'Not yet a statutory candidate',
  u'cycles': [2008, 2010, 2012],
  u'district': None,
  u'election_years': [2008, 2012],
  u'incumbent_challenge': u'I',
  u'incumbent_challenge_full': u'Incumbent',
  u'name': u'OBAMA, BARACK',
  u'office': u'P',
  u'office_full': u'President',
  u'party': u'DEM',
  u'party_full': u'Democratic Party',
  u'state': u'US'}]

In [0]:
candidates_2012_df = pd.DataFrame(candidates_2012)

In [0]:
candidates_2012_df.head()

,active_through,candidate_id,candidate_status,candidate_status_full,cycles,district,election_years,incumbent_challenge,incumbent_challenge_full,name,office,office_full,party,party_full,state
0,2012,S2UT00229,C,Statutory candidate,[2012],None,[2012],C,Challenger,"AALDERS, TIMOTHY NOEL",S,Senate,REP,Republican Party,UT
1,2012,H2CA01110,N,Not yet a statutory candidate,[2012],01,[2012],C,Challenger,"AANESTAD, SAMUEL",H,House,REP,Republican Party,CA
2,2012,H2MT00039,N,Not yet a statutory candidate,[2012],00,[2012],O,Open seat,"ABARR, JOHN ALLEN",H,House,DEM,Democratic Party,MT
3,2012,H2AZ02279,C,Statutory candidate,[2012],02,[2012],C,Challenger,"ABOUD, PAULA ANN",H,House,DEM,Democratic Party,AZ
4,2014,H4NY07011,N,Not yet a statutory candidate,"[1986, 1988, 1990, 1992, 1994, 1996, 1998, 200...",05,"[1984, 1986, 1988, 1990, 1992, 1994, 1996, 199...",None,Unknown,"ACKERMAN, GARY L.",H,House,DEM,Democratic Party,NY


This gives us something we can query:

In [0]:
candidates_2012_df[candidates_2012_df.name.str.match('romney|obama', case=False)].T

,2011,2331
active_through,2012,2012
candidate_id,P80003338,P80003353
candidate_status,N,C
candidate_status_full,Not yet a statutory candidate,Statutory candidate
cycles,"[2008, 2010, 2012]","[2008, 2010, 2012]"
district,None,None
election_years,"[2008, 2012]","[2008, 2012]"
incumbent_challenge,I,C
incumbent_challenge_full,Incumbent,Challenger
name,"OBAMA, BARACK","ROMNEY, MITT / PAUL D. RYAN"


In [0]:
candidates_2012_df[(candidates_2012_df.office == 'P') & (candidates_2012_df.candidate_status.str.match('C'))].name

104               BACHMANN, MICHELE
141           BARR, ROSEANNE CHERRI
333                 BROWN, HARLEY D
1186                HERMAN, RAPHAEL
1208            HILL, CHRISTOPHER V
1404                   KARGER, FRED
1614                  LINDSAY, PETA
1769           MCCALL, JAMES HATTON
1838                MESPLAY, KENT P
2203        RAKOWITZ, ARTHUR FABIAN
2331    ROMNEY, MITT / PAUL D. RYAN
2697          TERRY, RANDALL A. MR.
2877          WELLS, ROBERT CARR JR
2909      WIFORD, SAMUEL TIMOTHY II
Name: name, dtype: object

In [0]:
resp = requests.get('http://www.fec.gov/finance/disclosure/metadata/cn_header_file.csv')

In [0]:
cn_headers = resp.content.strip().split(',')

In [0]:
cn12 = pd.read_csv('_data/cn.txt', sep='|', header=None, names=cn_headers,
                   dtype={'CAND_OFFICE_DISTRICT':np.object, 'CAND_ELECTION_YR':np.object})

In [0]:
cn_headers

['CAND_ID',
 'CAND_NAME',
 'CAND_PTY_AFFILIATION',
 'CAND_ELECTION_YR',
 'CAND_OFFICE_ST',
 'CAND_OFFICE',
 'CAND_OFFICE_DISTRICT',
 'CAND_ICI',
 'CAND_STATUS',
 'CAND_PCC',
 'CAND_ST1',
 'CAND_ST2',
 'CAND_CITY',
 'CAND_ST',
 'CAND_ZIP']

In [0]:
map_fieldnames = {
    'CAND_PTY_AFFILIATION': 'party',
    'CAND_ELECTION_YR': 'cycle',
    'CAND_OFFICE_ST': 'state',
    'CAND_OFFICE': 'office',
    'CAND_OFFICE_DISTRICT': 'district',
    'CAND_ICI': 'incumbent_challenge',
    'CAND_STATUS': 'candidate_status',
    'CAND_NAME': 'name',
    'CAND_ID': 'candidate_id',
    'CAND_PCC': 'principal_committee',
}

In [0]:
cn12_rename = cn12.rename(columns=map_fieldnames)

In [0]:
comparison = cn12_rename.set_index('candidate_id').join(
    candidates_2012_df.set_index('candidate_id'),
    how='left', lsuffix='__CSV', rsuffix='__API')

In [0]:
comparison.office__API.value_counts()

H    2066
P     427
S     423
dtype: int64

In [0]:
comparison.office__CSV[comparison.cycle == '2012'].value_counts()

H    2595
S     481
P     438
dtype: int64

In [0]:
comparison[comparison.office__API.notnull()].shape

(2916, 28)

In [0]:
comparison[(comparison.candidate_status__API.notnull()) & (comparison.candidate_status__API != comparison.candidate_status__CSV)].shape

(1281, 28)

In [0]:
comparison[(comparison.office__API.notnull()) & (comparison.office__API != comparison.office__CSV)].shape

(0, 28)

In [0]:
comparison[comparison.name__API.isnull()].district__CSV.value_counts()

00    689
02    243
01    242
03    218
04    160
05    138
07    114
08    102
06     98
09     65
13     63
10     63
11     46
12     42
17     33
14     33
19     31
20     26
23     24
24     22
16     20
18     20
15     18
25     16
27     14
28     13
21     13
22     12
26     11
29     10
36     10
33      9
32      9
30      9
47      8
37      7
50      6
31      5
52      5
35      4
45      4
34      4
48      4
39      3
46      3
42      3
51      3
53      3
38      2
43      2
41      2
49      1
44      1
40      1
dtype: int64

In [0]:
nonmatches = comparison[comparison.name__API.isnull()]

In [0]:
nonmatches.pivot_table(index='district__CSV', columns='office__CSV', values='name__CSV', aggfunc=np.size)

office__CSV,H,P,S
district__CSV,,,
00,70,181,438
01,242,NaN,NaN
02,243,NaN,NaN
03,218,NaN,NaN
04,160,NaN,NaN
05,138,NaN,NaN
06,98,NaN,NaN
07,114,NaN,NaN
08,102,NaN,NaN


In [0]:
cn12_rename[cn12_rename.district == "99"]

,candidate_id,name,party,cycle,state,office,district,incumbent_challenge,candidate_status,principal_committee,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP


In [0]:
comparison.pivot_table(index='district__CSV', columns='office__CSV', values='name__CSV', aggfunc=np.size)

office__CSV,H,P,S
district__CSV,,,
00,129,608,861
01,423,NaN,NaN
02,468,NaN,NaN
03,366,NaN,NaN
04,321,NaN,NaN
05,258,NaN,NaN
06,238,NaN,NaN
07,223,NaN,NaN
08,201,NaN,NaN


In [0]:
[x for x in vc.iterkv()]

[('N', 2356), ('C', 1884), ('P', 1329), ('F', 58)]

In [0]:
for f in limited_vocab_fields:
    for v,c in cn12[f].value_counts().iterkv():
        print (f,v,c)

CAND_PTY_AFFILIATION REP 2477
CAND_PTY_AFFILIATION DEM 1879
CAND_PTY_AFFILIATION IND 394
CAND_PTY_AFFILIATION LIB 239
CAND_PTY_AFFILIATION OTH 152
CAND_PTY_AFFILIATION NNE 115
CAND_PTY_AFFILIATION GRE 112
CAND_PTY_AFFILIATION UNK 105
CAND_PTY_AFFILIATION DFL 22
CAND_PTY_AFFILIATION CON 20
CAND_PTY_AFFILIATION NPA 19
CAND_PTY_AFFILIATION REF 15
CAND_PTY_AFFILIATION CST 9
CAND_PTY_AFFILIATION IAP 7
CAND_PTY_AFFILIATION AIP 6
CAND_PTY_AFFILIATION IDP 6
CAND_PTY_AFFILIATION CIT 5
CAND_PTY_AFFILIATION SWP 5
CAND_PTY_AFFILIATION SUS 4
CAND_PTY_AFFILIATION CRV 3
CAND_PTY_AFFILIATION PAF 3
CAND_PTY_AFFILIATION W 3
CAND_PTY_AFFILIATION AMP 2
CAND_PTY_AFFILIATION TX 2
CAND_PTY_AFFILIATION IP 1
CAND_PTY_AFFILIATION HOU 1
CAND_PTY_AFFILIATION WFP 1
CAND_PTY_AFFILIATION N 1
CAND_PTY_AFFILIATION NLP 1
CAND_PTY_AFFILIATION RTL 1
CAND_PTY_AFFILIATION TEA 1
CAND_PTY_AFFILIATION PG 1
CAND_PTY_AFFILIATION FED 1
CAND_PTY_AFFILIATION UST 1
CAND_PTY_AFFILIATION DCG 1
CAND_PTY_AFFILIATION PRI 1
CAND_ELECTION

58

In [0]:
for f in limited_vocab_fields:
    

In [0]:
cn12[(cn12.CAND_STATUS == 'C') & (cn12.CAND_ELECTION_YR == 2012)].shape[0]

1857

## FEC Data Quality

Let's say we're interested in presidential candidates in the 2016 election. Using the `/candidates` (note the plural) endpoint, we can specify:

In [0]:
get_2016_presidential = {
    "cycle": 2016,
    "office": "P",  # for President. Also available: H(ouse) and S(enate)
}

presidential_candidates_2016 = [r for r in all_results("/candidates/search", params=get_2016_presidential)]

INFO:root:querying endpoint: http://api.open.fec.gov/v1/candidates/search
INFO:root:19 pages to be retrieved, with 373 records
INFO:root:all pages retrieved


To clarify, the full url we're accessing is:

    http://api.open.fec.gov/v1/candidates/search?office=P&cycle=2016&api_key=<<YOUR API KEY>>

In [0]:
len(presidential_candidates_2016)

373

Wow, that's a lot of presidential candidates! The republican primary is crowded, for sure, but 373 is a lot. Were there this many last time around?

In [0]:
get_2012_presidential = {
    "cycle": 2012,
    "office": "P",  # for President. Also available: H(ouse) and S(enate)
}

count_results("/candidates/search", params=get_2012_presidential)

440

In [0]:
get_2016_presidential_

In [0]:
presidential_candidates_2012 = [r for r in all_results("/candidates/search", params=get_2012_presidential)]

INFO:root:querying endpoint: http://api.open.fec.gov/v1/candidates/search
INFO:root:22 pages to be retrieved, with 440 records
INFO:root:all pages retrieved


In [0]:
cids_2016 = set(p['candidate_id'] for p in presidential_candidates_2016)
cids_2012 = set(p['candidate_id'] for p in presidential_candidates_2012)

In [0]:
len(set(cids_2012) & set(cids_2016))

83

Apparently not.

To explain what's happening

In [0]:
[pc for pc in presidential_candidates if 'RUBIO' in pc['name']]

[{u'active_through': 2016,
  u'candidate_id': u'P60006723',
  u'candidate_status': u'N',
  u'candidate_status_full': u'Not yet a statutory candidate',
  u'cycles': [2016],
  u'district': None,
  u'election_years': [2016],
  u'incumbent_challenge': u'O',
  u'incumbent_challenge_full': u'Open seat',
  u'name': u'RUBIO, MARCO',
  u'office': u'P',
  u'office_full': u'President',
  u'party': u'REP',
  u'party_full': u'Republican Party',
  u'principal_committees': [{u'candidate_ids': [u'P60006723', u'S0FL00338'],
    u'committee_id': u'C00458844',
    u'committee_type': u'P',
    u'committee_type_full': u'Presidential',
    u'cycles': [2010, 2012, 2014, 2016],
    u'designation': u'P',
    u'designation_full': u'Principal campaign committee',
    u'expire_date': None,
    u'first_file_date': u'2009-02-05T00:00:00+00:00',
    u'last_file_date': u'2015-04-13T00:00:00+00:00',
    u'name': u'MARCO RUBIO FOR PRESIDENT',
    u'organization_type': None,
    u'organization_type_full': None,
    u'pa

In [0]:
[pc for pc in presidential_candidates if 'RUBIO' in pc['name']]

[{u'active_through': 2016,
  u'candidate_id': u'P60006723',
  u'candidate_status': u'N',
  u'candidate_status_full': u'Not yet a statutory candidate',
  u'cycles': [2016],
  u'district': None,
  u'election_years': [2016],
  u'incumbent_challenge': u'O',
  u'incumbent_challenge_full': u'Open seat',
  u'name': u'RUBIO, MARCO',
  u'office': u'P',
  u'office_full': u'President',
  u'party': u'REP',
  u'party_full': u'Republican Party',
  u'state': u'US'}]

In [0]:
requests.get(BASE_URL+"/committee/C00458844").json()

{u'api_version': u'1.0',
 u'pagination': {u'count': 1, u'page': 1, u'pages': 1, u'per_page': 20},
 u'results': [{u'candidate_ids': [u'P60006723', u'S0FL00338'],
   u'city': u'MIAMI',
   u'committee_id': u'C00458844',
   u'committee_type': u'P',
   u'committee_type_full': u'Presidential',
   u'custodian_city': None,
   u'custodian_name_1': None,
   u'custodian_name_2': None,
   u'custodian_name_full': None,
   u'custodian_name_middle': None,
   u'custodian_name_prefix': None,
   u'custodian_name_suffix': None,
   u'custodian_name_title': None,
   u'custodian_phone': None,
   u'custodian_state': None,
   u'custodian_street_1': None,
   u'custodian_street_2': None,
   u'custodian_zip': None,
   u'cycles': [2010, 2012, 2014, 2016],
   u'designation': u'P',
   u'designation_full': u'Principal campaign committee',
   u'email': u'LLISKER@HDAFEC.COM',
   u'expire_date': None,
   u'fax': None,
   u'filing_frequency': u'Q',
   u'first_file_date': u'2009-02-05T00:00:00+00:00',
   u'form_type': u'

### Candidates

Many of the `/candidate` endpoints require that we know the FEC ID that is assigned to the candidate.  You can search

In [0]:
res = requests.get('/candidates?sort=name&per_page=20&page=1')

In [0]:
data = res.json()

In [0]:
data['pagination']

{u'count': 20281, u'page': 1, u'pages': 1015, u'per_page': 20}

In [0]:
data['api_version']

u'1.0'

In [0]:
data['results']

[{u'active_through': 2004,
  u'candidate_id': u'P40002172',
  u'candidate_status': u'N',
  u'candidate_status_full': u'Not yet a statutory candidate',
  u'cycles': [2004],
  u'district': None,
  u'election_years': [2004],
  u'incumbent_challenge': None,
  u'incumbent_challenge_full': u'Unknown',
  u'name': u'AABBATTE, MICHAEL THOMAS WITORT',
  u'office': u'P',
  u'office_full': u'President',
  u'party': u'IND',
  u'party_full': u'Independent',
  u'state': u'US'},
 {u'active_through': 2012,
  u'candidate_id': u'S2UT00229',
  u'candidate_status': u'N',
  u'candidate_status_full': u'Not yet a statutory candidate',
  u'cycles': [2012],
  u'district': None,
  u'election_years': [2012],
  u'incumbent_challenge': None,
  u'incumbent_challenge_full': u'Unknown',
  u'name': u'AALDERS, TIMOTHY NOEL',
  u'office': u'S',
  u'office_full': u'Senate',
  u'party': u'REP',
  u'party_full': u'Republican Party',
  u'state': u'UT'},
 {u'active_through': 2012,
  u'candidate_id': u'H2CA01110',
  u'candidat

In [0]:
state_query = 'http://fec-dev-api.cf.18f.us/v1/candidates?sort=name&candidate_status=C&office=H&state={}&cycle=2012&per_page=20&page=1'

for state in ('PA', 'NY', 'NJ'):
    num_results = requests.get(state_query.format(state)).json()['pagination']['count']
    print(state,num_results)
    

('PA', 17)
('NY', 37)
('NJ', 14)


In [0]:
requests.get(state_query.format('PA,NY,NJ')).json()['pagination']['count']

0

# Sunlight from the inside out

The group of developers working on OpenFEC includes two former Sunlight labs members. We couldn't be prouder of the work they've been doing during their time "on the inside." It's unsurprising, though, that they've been effective at 18f and more specifically on the OpenFEC project. Lindsay Young developed our portal for accessing a live feed Foreign Agent Registration Act, and Alison Rowland was my predecessor as project lead on Influence Explorer. We miss them both, but we're very grateful for the hard work they and their team are putting into improving public access to campaign finance disclosure at the federal level.